In [1]:
import pandas as pd
import numpy as np
import os
import json
import gspread as gs
import warnings

from clean_argy import argy_wrangling
from top_imports import top_importadores
from data_anual import annual_data

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

warnings.simplefilter("ignore")

                ### ~~~ ###
            ###     PATH     ###
                ### ~~~ ###

location_2013 = "./data/283525_Importaciones_2013.xlsx"
location_2014 = "./data/283525_Importaciones_2014.xlsx"
location_2015 = "./data/283525_Importaciones_2015.xlsx"
location_2016 = "./data/283525_Importaciones_2016.xlsx"
location_2017 = "./data/283525_Importaciones_2017.xlsx"
location_2018 = "./data/283525_Importaciones_2018.xlsx"
location_2019 = "./data/283525_Importaciones_2019.xlsx"
location_2020 = "./data/283525_Importaciones_2020.xlsx"
location_2021 = "./data/283525_Importaciones_2021.xlsx"
location_2022 = "./data/283525_Importaciones_2022.xlsx"
location_2023 = "./data/283525_Importaciones_2023.xlsx"

                ### ~~~ ###
            ###     DATA     ###
                ### ~~~ ###

data2013 = pd.read_excel(location_2013, engine="openpyxl")
data2014 = pd.read_excel(location_2014, engine="openpyxl")
data2015 = pd.read_excel(location_2015, engine="openpyxl")
data2016 = pd.read_excel(location_2016, engine="openpyxl")
data2017 = pd.read_excel(location_2017, engine="openpyxl")
data2018 = pd.read_excel(location_2018, engine="openpyxl")
data2019 = pd.read_excel(location_2019, engine="openpyxl")
data2020 = pd.read_excel(location_2020, engine="openpyxl")
data2021 = pd.read_excel(location_2021, engine="openpyxl")
data2022 = pd.read_excel(location_2022, engine="openpyxl")
data2023 = pd.read_excel(location_2023, engine="openpyxl")



In [2]:
# MERGE
dfs = [data2013, data2014, data2015, data2016, data2017, data2018, data2019, data2020, data2021, data2022, data2023]

print("Este df tiene (filas, col):\n___________________")

for df in dfs:
    print(f"> {df['Fecha'][0].year}: {df.shape}\n~~~~~~~~~~~~~~~~~~~")
    


Este df tiene (filas, col):
___________________
> 2013: (84, 35)
~~~~~~~~~~~~~~~~~~~
> 2014: (99, 35)
~~~~~~~~~~~~~~~~~~~
> 2015: (79, 35)
~~~~~~~~~~~~~~~~~~~
> 2016: (75, 35)
~~~~~~~~~~~~~~~~~~~
> 2017: (66, 35)
~~~~~~~~~~~~~~~~~~~
> 2018: (70, 35)
~~~~~~~~~~~~~~~~~~~
> 2019: (70, 35)
~~~~~~~~~~~~~~~~~~~
> 2020: (63, 35)
~~~~~~~~~~~~~~~~~~~
> 2021: (67, 35)
~~~~~~~~~~~~~~~~~~~
> 2022: (71, 35)
~~~~~~~~~~~~~~~~~~~
> 2023: (9, 35)
~~~~~~~~~~~~~~~~~~~


In [3]:
results_dfs = argy_wrangling(dfs)

# df_final = pd.concat(dfs, axis=0)
# df_final.isnull().sum()
# df_final.sample(25)


~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping nulls...
> Done with: 2013
~~~~~~~~~~~~~~~~~~~
~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping nulls...
> Done with: 2014
~~~~~~~~~~~~~~~~~~~
~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping nulls...
> Done with: 2015
~~~~~~~~~~~~~~~~~~~
~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping nulls...
> Done with: 2016
~~~~~~~~~~~~~~~~~~~
~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping nulls...
> Done with: 2017
~~~~~~~~~~~~~~~~~~~
~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping nulls...
> Done with: 2018
~~~~~~~~~~~~~~~~~~~
~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping nulls...
> Done with: 2019
~~~~~~~~~~~~~~~~~~~
~ Limpiando marcas...
~ Limpiando NCMs...
~ Creando columna de precio...
~ Dropping

In [4]:
print(f"> Estadísticos del precio en el año:\n___________________")

for df in results_dfs:
    print(f"- En el {df['Fecha'][5].year}:\n{df['U$S Unitario'].describe()}\n~~~~~~~~~~~~~~~~~~~")

> Estadísticos del precio en el año:
___________________
- En el 2013:
count    71.000000
mean      3.300704
std       2.783427
min       0.450000
25%       0.665000
50%       3.460000
75%       4.600000
max      17.910000
Name: U$S Unitario, dtype: float64
~~~~~~~~~~~~~~~~~~~
- En el 2014:
count    86.000000
mean      1.963372
std       1.967647
min       0.500000
25%       0.550000
50%       0.700000
75%       3.470000
max       9.090000
Name: U$S Unitario, dtype: float64
~~~~~~~~~~~~~~~~~~~
- En el 2015:
count      69.000000
mean       28.147971
std       209.755511
min         0.340000
25%         0.620000
50%         2.880000
75%         4.390000
max      1745.050000
Name: U$S Unitario, dtype: float64
~~~~~~~~~~~~~~~~~~~
- En el 2016:
count    58.000000
mean      2.951724
std       1.836126
min       0.360000
25%       0.732500
50%       3.330000
75%       3.897500
max       7.450000
Name: U$S Unitario, dtype: float64
~~~~~~~~~~~~~~~~~~~
- En el 2017:
count    53.000000
mean      

In [5]:
annual_data(results_dfs)


- 2013 appended.
- 2014 appended.
- 2015 appended.
- 2016 appended.
- 2017 appended.
- 2018 appended.
- 2019 appended.
- 2020 appended.
- 2021 appended.
- 2022 appended.
- 2023 appended.
~~~~~~~~~~~~~~~~~~~
> Transition dictionary:
- NCM: ['283525', '283525', '283525', '283525', '283525', '283525', '283525', '283525', '283525', '283525', '283525']
- Año: [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
- Volumen Total: [1046.98, 2113.66, 1603.85, 1009.04, 1093.95, 1176.1, 953.18, 368.02, 424.08, 511.58, 32.0]


In [6]:
top_importadores(results_dfs)


- Done with: IVAX ARGENTINA SOCIEDAD ANONIMA (2013) 
- Done with: M.CASSAB ARGENTINA S A (2013) 
- Done with: OXAR S.A. (2013) 
- Done with: BRASCORP S.A. (2013) 
- Done with: ENSOL S A (2013) 
- Done with: TREND CHEMICAL SRL (2013) 
- Done with: BAYER SOCIEDAD ANONIMA (2013) 
- Done with: PREST S A C I Y A (2013) 
- Done with: PRODUCTOS DESTILADOS S A C I Y F (2013) 
- Done with: CATALENT ARGENTINA SOCIEDAD ANONIMA INDU (2013) 
- Done with: ETILFARMA S A (2013) 
- Done with: ABBOTT LABORATORIES ARGENTINA SA (2013) 
- Done with: LABORATORIO SCOPE SOCIEDAD ANONIMA (2013) 
- Done with: DUHALDE Y CIA  SOCIEDAD DE RESPONSABILID (2013) 
- Done with: FARMATRADE S R L (2013) 
- Done with: NUTRIPHOS S.A. (2013) 
- Done with: GLAXOSMITHKLINE ARGENTINA SOCIEDAD ANONI (2013) 
- Done with: FACYT S R L (2013) 
- Done with: BRASCORP S.A. (2014) 
- Done with: LABORATORIOS BETA SOCIEDAD ANONIMA (2014) 
- Done with: PRODUCTOS DESTILADOS S A C I Y F (2014) 
- Done with: BAYER SOCIEDAD ANONIMA (2014) 
- 

,Año,Importador,Participacion,Precio Promedio,Volumen Total (TN)
3,2013,BRASCORP S.A.,58%,610.0,603.84
4,2013,ENSOL S A,10%,540.0,100.00
6,2013,BAYER SOCIEDAD ANONIMA,9%,3470.0,92.00
0,2014,BRASCORP S.A.,73%,590.0,1552.03
10,2014,ALIMENTAL S A,11%,500.0,240.00
3,2014,BAYER SOCIEDAD ANONIMA,5%,3480.0,111.00
0,2015,BRASCORP S.A.,62%,630.0,999.04
17,2015,ASOCIACION DE COOPERATIVAS ARGENTINAS CO,19%,540.0,300.00
1,2015,BAYER SOCIEDAD ANONIMA,7%,3430.0,108.00
0,2016,ASOCIACION DE COOPERATIVAS ARGENTINAS CO,40%,590.0,400.00


In [7]:
# EXPORT TO PICKLE
CURRENT_WD = os.getcwd()
COUNTRY_WD = os.path.basename(CURRENT_WD).split('/')[-1]

# df_final.to_pickle(f'{COUNTRY_WD}.pkl')
